In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

from azure.ai.ml import dsl, Input, Output
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION="a3f56f48-3efb-4970-81a3-e4eda598333c"
RESOURCE_GROUP="luiz.victor.dev-rg"
WS_NAME="tcc-experiments"

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [ ]:
TRAIN_DATAS = [
    "vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017.csv"
]

TEST_DATAS = [
    "vrex_encoded_tf_idf_2018_2019_2020_2021.csv"
]

version = "v1"

arr_data_to_train = []
arr_data_to_test = []

for to_train, to_test in zip(TRAIN_DATAS, TEST_DATAS):
    data_to_train = ml_client.data.get(name=to_train.split(".")[0], version=version)
    arr_data_to_train.append(data_to_train)
    print(f"Data to train asset URI: {data_to_train.path} - name: {to_train.split('.')[0]}")

    data_to_test = ml_client.data.get(name=to_test.split('.')[0], version=version)
    arr_data_to_test.append(data_to_test)
    print(f"Data to test asset URI: {data_to_test.path} - name: {to_test.split('.')[0]}")

In [ ]:
# importing the Component Package
from azure.ai.ml import load_component

data_prep_component = load_component(source="/home/azureuser/cloudfiles/code/Users/luiz.victor.dev/tcc_experiments_az_ml/src/components/preparation/data_prep.yaml")
data_prep_component = ml_client.create_or_update(data_prep_component)

gini_component = load_component(source="/home/azureuser/cloudfiles/code/Users/luiz.victor.dev/tcc_experiments_az_ml/src/components/feature_selection/gini.yaml")
gini_component = ml_client.create_or_update(gini_component)

train_gbc = load_component(source="/home/azureuser/cloudfiles/code/Users/luiz.victor.dev/tcc_experiments_az_ml/src/components/train/train_gbc.yaml")
train_gbc = ml_client.create_or_update(train_gbc)

train_nbc = load_component(source="/home/azureuser/cloudfiles/code/Users/luiz.victor.dev/tcc_experiments_az_ml/src/components/train/train_nbc.yaml")
train_nbc = ml_client.create_or_update(train_nbc)

train_rfc = load_component(source="/home/azureuser/cloudfiles/code/Users/luiz.victor.dev/tcc_experiments_az_ml/src/components/train/train_rfc.yaml")
train_rfc = ml_client.create_or_update(train_rfc)

train_svc = load_component(source="/home/azureuser/cloudfiles/code/Users/luiz.victor.dev/tcc_experiments_az_ml/src/components/train/train_svc.yaml")
train_svc = ml_client.create_or_update(train_svc)

train_xgb = load_component(source="/home/azureuser/cloudfiles/code/Users/luiz.victor.dev/tcc_experiments_az_ml/src/components/train/train_xgb.yaml")
train_xgb = ml_client.create_or_update(train_xgb)

In [ ]:
TOTAL_TRIALS = 20
CONCURRENT_TRIALS = 10
TIMEOUT = 2*3600
TIMEOUT_PLUS = 2*7200

COMPUTE = 'serverless'
SAMPLING_ALGORITHM = 'bayesian'
METRIC = 'training_accuracy_score'
GOAL = 'Maximize'
EVALUATION_INTERVAL = 1
DELAY_EVALUATION = 5

# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline


In [ ]:
@dsl.pipeline(
    compute="serverless",
    description="Feature Selection with Gini and train with GBC",
)
def train_gbc_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    gini_job = gini_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_percentage=0.5,
    )

    train_gbc_job = train_gbc(
        train_data=gini_job.outputs.train_data_feat_sel,  
        test_data=gini_job.outputs.test_data_feat_sel,  
        n_estimators_to_gbc=Choice(values=[50, 100, 200]),
        learning_rate_to_gbc=Uniform(min_value=0.01, max_value=0.3),
    )

    sweep_step_to_gbc = train_gbc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_gbc.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT)

    sweep_step_to_gbc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)


In [ ]:
@dsl.pipeline(
    compute="serverless",
    description="Feature Selection with Gini and train with NBC",
)
def train_nbc_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    gini_job = gini_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_percentage=0.5,
    )

    train_nbc_job = train_nbc(
        train_data=gini_job.outputs.train_data_feat_sel,  
        test_data=gini_job.outputs.test_data_feat_sel,   
    )

In [ ]:
@dsl.pipeline(
    compute="serverless",
    description="Feature Selection with Gini and train with RFC",
)
def train_rfc_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    gini_job = gini_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_percentage=0.5,
    )

    train_rfc_job = train_rfc(
        train_data=gini_job.outputs.train_data_feat_sel,  
        test_data=gini_job.outputs.test_data_feat_sel, 
        n_estimators_to_rfc=Choice(values=[50, 100, 200]),
    )

    sweep_step_to_rfc = train_rfc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_rfc.set_limits(max_total_trials=TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT)
    sweep_step_to_rfc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [ ]:
@dsl.pipeline(
    compute="serverless",
    description="Feature Selection with Gini and train with SVC",
)
def train_svc_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    gini_job = gini_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_percentage=0.5,
    )

    train_svc_job = train_svc(
        train_data=gini_job.outputs.train_data_feat_sel,  
        test_data=gini_job.outputs.test_data_feat_sel,   
        kernel_to_svc=Choice(values=["linear", "rbf", "poly", "sigmoid","precomputed"]),
        gamma_to_svc=Choice(values=["scale", "auto"]),
    )

    sweep_step_to_svc = train_svc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_svc.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT_PLUS)
    sweep_step_to_svc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [ ]:
@dsl.pipeline(
    compute="serverless",
    description="Feature Selection with Gini and train with XGB",
)
def train_xgb_pipeline(
    data_to_train,
    data_to_test,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    gini_job = gini_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_percentage=0.5,
    )

    train_xgb_job = train_xgb(
        train_data=gini_job.outputs.train_data_feat_sel,  
        test_data=gini_job.outputs.test_data_feat_sel,   
        n_estimators_to_xgb=Choice(values=[100, 500, 1000]),
        learning_rate_to_xgb=Uniform(min_value=0.01, max_value=0.3),
    )

    sweep_step_to_xgb = train_xgb_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_xgb.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT_PLUS)
    sweep_step_to_xgb.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [ ]:
pipelines = []

for data_to_train, data_to_test in zip(arr_data_to_train, arr_data_to_test):

    pipelines.append(train_nbc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_gbc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_rfc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_svc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_xgb_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))


In [ ]:
import datetime as dt

models = ['nbc', 'gbc', 'rfc', 'svc', 'xgb']

def _get_experiment_names() -> [str]:
    experiment_names = []
    for train_name, test_name in zip(TRAIN_DATAS, TEST_DATAS):
        for model_name in models:
            current_time = dt.datetime.now()
            formatted_time = current_time.strftime("%Y_%m_%d_%H_%M_%S")  # Formata a data e hora atual
            train_name_base = train_name.split('.')[0]
            test_name_base = test_name.split('.')[0]
            name = f"{train_name_base}_tested_{test_name_base}_w_model_{model_name}_w_encoded_tfidf_data_w_infogain_feat_sel"
            experiment_names.append(name)
            print(name)
    return experiment_names


In [ ]:
experiment_names = _get_experiment_names()
for pipeline, experiment_name in zip(pipelines, experiment_names):
    pipeline_job = ml_client.jobs.create_or_update(
        pipeline,
        experiment_name=experiment_name,
    )
    ml_client.jobs.stream(pipeline_job.name)